In [ ]:
# Compare rates to tierd pricing

In [4]:
import pandas


Set data.  Default is a sample-hourly.xls
To analyze your own data - first download your data aian hourly format from the Hydro Ottawa billing/usage area in Excel format.  Then upload it on the left (up arrow button).  Change the name to match your file.

In [5]:
df = pandas.read_excel("yearbyhour.xls", header=3) # skips first 3 lines
#check output
df.head(2)

,Date,Time,Rate Type,Consumption (kWh),Cost ($)
0,2022-04-09,2022-04-09 00:00:00,Off-Peak,0.62,0.05
1,2022-04-09,2022-04-09 01:00:00,Off-Peak,0.61,0.05


### Compare Current Time of Day use to Tiered Pricing

In [6]:
df["Month"] = pandas.to_datetime(df['Time']).dt.strftime('%Y-%m')
out = df.groupby(by="Month", as_index=False).sum(["Consumption (kWh)", "Cost ($)"])
summer = [5,6,7,8,9,10]
winter = [1,2,3,4,11,12]

rate_low = 0.087
rate_high = 0.103
rate_summer_threshold = 600 #kwh
rate_winter_threshold = 1000 #kwh

def f(x):
    c = x["Consumption (kWh)"]
    mo = int(x["Month"].split("-")[1])
    if (mo in summer and c <= rate_summer_threshold):
        t = (c * rate_low)
        r = "summer low"
    elif (mo in summer and c > rate_summer_threshold):
        t = (rate_summer_threshold * rate_low) + (c - rate_summer_threshold)*rate_high
        r = "summer high"
    elif (mo in winter and c <= rate_winter_threshold):
        t = (c * rate_low)
        r = "winter low"
    elif (mo in winter and c > rate_winter_threshold):
        t = (rate_winter_threshold * rate_low) + (c - rate_winter_threshold)*rate_high
        r = "winter high"
    else:
        t = 0
        r = ""
    
    x["Tiered Price"] = round(t,2)
    x["Tiered Rate"] = r
    x["Difference with time of use"] = round(x["Cost ($)"] - t,2)
    return x

out = out.apply(f, axis=1)
out.head(14)

,Month,Consumption (kWh),Cost ($),Tiered Price,Tiered Rate,Difference with time of use
0,2022-04,497.40,51.49,43.27,winter low,8.22
1,2022-05,526.67,55.61,45.82,summer low,9.79
2,2022-06,706.47,75.79,63.17,summer high,12.62
3,2022-07,421.92,42.13,36.71,summer low,5.42
4,2022-08,884.89,89.65,81.54,summer high,8.11
5,2022-09,755.29,79.45,68.19,summer high,11.26
6,2022-10,800.47,81.53,72.85,summer high,8.68
7,2022-11,812.19,77.01,70.66,winter low,6.35
8,2022-12,869.28,80.79,75.63,winter low,5.16
9,2023-01,826.37,77.55,71.89,winter low,5.66
